In [3]:
import os
import json
import time
import requests
import urllib.parse
from IPython.display import Image, display

In [4]:
#config_file = 'config.json'
#key = json.load(open(config_file))['mdvhwX6Era11JlFA5Mtx9Lar7xnzUxQc_6fuvtQ8p_w']
subscriptionKey = "mdvhwX6Era11JlFA5Mtx9Lar7xnzUxQc_6fuvtQ8p_w"

In [5]:
addresstoAnalyze = "Hlavná 63 Staré Mesto, 040 01 Staré Mesto" #Analyzing the Catchment Area for the Starbucks Store (400, Pine Street, Seattle) in Westlake Center, Seattle. 
locationDetails = requests.get("https://atlas.microsoft.com/search/address/json?subscription-key={}&api-version=1.0&query={}".format(subscriptionKey,addresstoAnalyze)).json()
latlong=locationDetails["results"][0]["position"]
currentLocation=[0,0]
currentLocation[0]=latlong["lat"]
currentLocation[1]=latlong["lon"]

In [6]:
print("Lat Long for",addresstoAnalyze,"is",latlong)
walkingTime = 300 # this is 600secs. you can experiment with different values
cTraffic = "true" #you can test how far you can go in current traffic
tMode = "pedestrian" # you can change this to car, bus, pedestrian, bicycle and more. see http://bit.ly/2nGErtX for details 

Lat Long for Hlavná 63 Staré Mesto, 040 01 Staré Mesto is {'lat': 48.72187, 'lon': 21.25778}


In [7]:
routeRangeResponse = requests.get("https://atlas.microsoft.com/route/range/json?subscription-key={}&api-version=1.0&query={}&timeBudgetInSec={}&traffic={}&travelmode={}".format(subscriptionKey,str(currentLocation[0])+","+str(currentLocation[1]),walkingTime,cTraffic,tMode)).json()

In [8]:
routeRangeResponseF = json.dumps(routeRangeResponse, indent=4)
print(routeRangeResponseF)

{
    "formatVersion": "0.0.1",
    "reachableRange": {
        "center": {
            "latitude": 48.72184,
            "longitude": 21.25766
        },
        "boundary": [
            {
                "latitude": 48.72452,
                "longitude": 21.25764
            },
            {
                "latitude": 48.72413,
                "longitude": 21.25736
            },
            {
                "latitude": 48.72403,
                "longitude": 21.25706
            },
            {
                "latitude": 48.72389,
                "longitude": 21.25663
            },
            {
                "latitude": 48.72529,
                "longitude": 21.25561
            },
            {
                "latitude": 48.72412,
                "longitude": 21.2559
            },
            {
                "latitude": 48.72316,
                "longitude": 21.25631
            },
            {
                "latitude": 48.72311,
                "longitude": 21.25598

In [9]:
polyBounds = routeRangeResponse["reachableRange"]["boundary"]


for i in range(len(polyBounds)):
    coordList = list(polyBounds[i].values())
    coordList[0], coordList[1] = coordList[1], coordList[0]
    polyBounds[i] = coordList

polyBounds.pop()
polyBounds.append(polyBounds[0])

boundsData = {
               "geometry": {
                 "type": "Polygon",
                 "coordinates": 
                   [
                      polyBounds
                   ]
                }
             }

In [10]:
boundsDataF = json.dumps(boundsData, indent=4)
print(boundsDataF)

{
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    21.25764,
                    48.72452
                ],
                [
                    21.25736,
                    48.72413
                ],
                [
                    21.25706,
                    48.72403
                ],
                [
                    21.25663,
                    48.72389
                ],
                [
                    21.25561,
                    48.72529
                ],
                [
                    21.2559,
                    48.72412
                ],
                [
                    21.25631,
                    48.72316
                ],
                [
                    21.25598,
                    48.72311
                ],
                [
                    21.25578,
                    48.72307
                ],
                [
                    21.25436,
   

In [11]:
poiName = "beer"
searchPolyResponse = requests.post(url = "https://atlas.microsoft.com/search/geometry/json?subscription-key={}&api-version=1.0&query={}&idxSet=POI&limit=50".format(subscriptionKey,poiName), json = boundsData).json()


In [12]:
numRes = searchPolyResponse["summary"]["numResults"]
print("Total Number of "+poiName+" nearby: ", numRes)

Total Number of beer nearby:  3


In [13]:
print("Here is a list of the Search Results: ")
for loc in range(len(searchPolyResponse["results"])):
                print(loc+1,". ",searchPolyResponse["results"][loc]["poi"]["name"])
                

Here is a list of the Search Results: 
1 .  Korzo Kráľovský pivovar
2 .  Luxor Brewhouse
3 .  Beer House


In [14]:
searchLocations = []
for loc in range(len(searchPolyResponse["results"])):
                location = list(searchPolyResponse["results"][loc]["position"].values())
                location[0], location[1] = location[1], location[0]
                searchLocations.append(location)

In [ ]:
rangeData = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          polyBounds
        ]
      }
    }
  ]
}

# Upload range data to Azure Maps data service.
uploadRangeResponse = requests.post(url = "https://atlas.microsoft.com/mapData/upload?subscription-key={}&api-version=1.0&dataFormat=geojson".format(subscriptionKey), json = rangeData)


rangeUdidRequest = uploadRangeResponse.headers["Location"]+"&subscription-key="+subscriptionKey

while True:
    getRangeUdid = requests.get(rangeUdidRequest).json()
    if 'udid' in getRangeUdid:
        break
    else:
        time.sleep(0.2)
rangeUdid = getRangeUdid["udid"]

In [ ]:
poiData = {
    "type": "FeatureCollection",
    "features": [
      {
        "type": "Feature",
        "properties": {},
        "geometry": {
            "type": "MultiPoint",
            "coordinates": searchLocations
        }
    }
  ]
}

uploadPOIsResponse = requests.post(url = "https://atlas.microsoft.com/mapData/upload?subscription-key={}&api-version=1.0&dataFormat=geojson".format(subscriptionKey), json = poiData)

poiUdidRequest = uploadPOIsResponse.headers["Location"]+"&subscription-key="+subscriptionKey

while True:
    getPoiUdid = requests.get(poiUdidRequest).json()
    if 'udid' in getPoiUdid:
        break
    else:
        time.sleep(0.2)
poiUdid = getPoiUdid["udid"]

In [ ]:
# Get bounds for bounding box.
def getBounds(polyBounds):
    maxLon = max(map(lambda x: x[0], polyBounds))
    minLon = min(map(lambda x: x[0], polyBounds))

    maxLat = max(map(lambda x: x[1], polyBounds))
    minLat = min(map(lambda x: x[1], polyBounds))
    
    lonBuffer = (maxLon-minLon)
    minLon -= lonBuffer
    maxLon += lonBuffer

    latBuffer = (maxLat-minLat)
    minLat -= latBuffer
    maxLat += latBuffer
    
    return [minLon, maxLon, minLat, maxLat]

minLon, maxLon, minLat, maxLat = getBounds(polyBounds)

path = "lcff3333|lw3|la0.80|fa0.35||udid-{}".format(rangeUdid)
pins = "default|an15 53||udid-{}".format(poiUdid)

encodedPins = urllib.parse.quote(pins, safe='')


staticMapResponse = requests.get("https://atlas.microsoft.com/map/static/png?api-version=1.0&subscription-key={}&pins={}&path={}&bbox={}&zoom=14".format(subscriptionKey,encodedPins,path,str(minLon)+", "+str(minLat)+", "+str(maxLon)+", "+str(maxLat)))


poiRangeMap = staticMapResponse.content

display(Image(poiRangeMap))